In [96]:
import pandas as pd
import numpy as np
import os.path
#pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [11]:
def merge_forcage_piezo(forcage_file, merge_file ='dataset/static_attributes/piezo_characs.csv', output_file = None):
    df_forcage =  pd.read_csv('dataset/time_series/forçages/' + forcage_file + '.txt', 
                  sep=';', 
                  skiprows=85,
                  comment='#')
    df_forcage.columns = df_forcage.columns.str.strip()
    df_piezo_charachs =  pd.read_csv(merge_file,sep=';',encoding='latin-1')
    
    bss_id = df_piezo_charachs[df_piezo_charachs['limnit'] == forcage_file ]['BSS_ID'].values[0]
    
    df_piezo = pd.read_csv("dataset/time_series/piezos/"+bss_id+".csv")
    
    df_piezo.head()

    df_piezo["date_mesure"] = (
        pd.to_datetime(df_piezo["date_mesure"])
        .dt.strftime("%Y%m%d")
        .astype("Int64")
    )

    df_piezo['Date'] = df_piezo['date_mesure'].astype('int64')
    #garder que les colonnes 'date_mesure' et 'niveau_nappe_eau'
    df_piezo = df_piezo[['Date', 'niveau_nappe_eau']]
    #join df_forcage et df_piezo sur la colonne 'date'
    df_merged = pd.merge(df_forcage, df_piezo, on='Date', how='inner')

    if(output_file != None):
        df_merged.to_csv(output_file, index=False)
        
    return df_merged
    
merge_forcage_piezo('A235020002')

,Date,Qls,Qmmj,Val_MD,Val_S,Val_Q,Val_M,Val_C,Val_I,Ptot,Fsol,Temp,E_OU,E_PE,E_PM,Vent,Humi,DLI,SSI,IHGR,SWI,SWE,TN,TX,niveau_nappe_eau
0,19761229,1950,0.608,-99,16,20,10,0,1,0.1,0.00,-7.1,0.0,0.1,0.0,1.5,2.1,2185.9,266.1,0.729,0.995,7.59,-7.3,-6.4,177.47
1,19770105,1900,0.592,-99,16,20,10,0,1,0.0,0.00,-1.4,0.1,0.0,0.0,1.8,3.2,2215.9,340.7,0.734,1.001,7.60,-2.2,-0.9,177.37
2,19770112,3010,0.938,-99,16,20,10,0,1,0.1,0.00,-2.1,0.1,0.0,0.0,1.3,3.1,2193.9,97.8,0.745,0.991,22.10,-2.6,-1.4,177.11
3,19770119,2100,0.654,-99,16,20,10,0,1,6.9,0.97,-1.5,0.1,0.1,0.1,2.2,3.3,2525.7,89.2,0.762,0.983,46.88,-4.4,1.2,177.19
4,19770126,15900,4.954,-99,16,20,10,0,1,6.0,0.00,7.1,0.6,0.5,0.4,4.3,5.6,2663.9,258.8,0.795,1.095,18.75,6.4,8.5,177.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8364,20220727,-99,-99.000,-99,-99,-99,-99,-99,-99,0.0,0.00,18.7,3.8,4.9,4.2,2.0,8.3,2947.5,2221.9,0.252,0.231,0.00,15.8,21.9,176.51
8365,20220728,-99,-99.000,-99,-99,-99,-99,-99,-99,0.0,0.00,20.4,4.1,4.5,3.7,2.7,8.4,2810.1,1962.1,0.242,0.226,0.00,15.1,25.5,176.50
8366,20220729,-99,-99.000,-99,-99,-99,-99,-99,-99,0.0,0.00,20.4,4.1,2.8,2.5,1.8,8.6,2797.5,1527.3,0.238,0.221,0.00,16.2,24.3,176.50
8367,20220730,-99,-99.000,-99,-99,-99,-99,-99,-99,0.0,0.00,20.9,4.1,3.8,3.4,1.7,8.1,2919.0,1805.7,0.231,0.216,0.00,16.6,25.4,176.49


In [82]:
def water_level_mean(bss_id):
    if type(bss_id) != str:
        bss_id = bss_id['BSS_ID']
    if os.path.isfile("dataset/time_series/piezos/"+str(bss_id)+".csv"):
        df_piezo = pd.read_csv("dataset/time_series/piezos/"+str(bss_id)+".csv")
        df_piezo.head()
        df_piezo["date_mesure"] = (
            pd.to_datetime(df_piezo["date_mesure"])
            .dt.strftime("%Y%m%d")
            .astype("Int64")
        )
        date_max = df_piezo["date_mesure"].astype('int64').max()
        return df_piezo[df_piezo["date_mesure"] > date_max - 40000]["niveau_nappe_eau"].mean()
    return pd.NaT
water_level_mean("BSS000KQJF")

154.76067077344285

In [132]:
def get_attributes():
    #Extract usefull data from piezo_characs
    raw_df_piezo_characs = pd.read_csv('dataset/static_attributes/piezo_characs.csv', sep=';', encoding='latin-1')
    df_piezo_characs = raw_df_piezo_characs[["BSS_ID", "H", "XL93", "YL93", "formation", "état", "nature", "milieu", "thème", "limnit", "rr_mean", "rr_p", "etp_mean", "stress_p"]].copy()
        
    df_piezo_characs["water_level_mean"] = df_piezo_characs.apply(water_level_mean, axis=1)
    df_piezo_characs.dropna(inplace=True)
    df_piezo_characs.reset_index(drop=True, inplace=True)
    df_piezo_characs.drop(columns=['BSS_ID', 'limnit'], inplace=True)

    df_len = len(df_piezo_characs.index)
    columns = []
    for col in list(df_piezo_characs.columns):
        if df_piezo_characs[col].value_counts(ascending=True).values[0] == df_len :
            columns.append(col)
    df_piezo_characs.drop(columns=columns, inplace=True)

    return df_piezo_characs

attribute = get_attributes()
attribute

,H,XL93,YL93,état,rr_mean,rr_p,etp_mean,stress_p,water_level_mean
0,31.00,8.402373e+05,6.336133e+06,nappe libre,3.372874,0.721424,2.019550,0.568147,25.903621
1,18.00,8.469111e+05,6.320550e+06,nappe libre,3.372874,0.721424,2.019550,0.568147,15.765496
2,37.00,8.387315e+05,6.343727e+06,nappe libre,2.292749,0.420474,2.922651,0.769507,31.184435
3,29.00,4.084455e+05,6.303154e+06,nappe libre et captive (spatialement),2.909801,0.637620,2.176989,0.638390,27.023546
4,22.58,8.466548e+05,6.309619e+06,nappe libre,2.569952,0.548554,2.406785,0.716975,18.456629
5,40.00,8.377316e+05,6.347091e+06,nappe libre,2.292749,0.420474,2.922651,0.769507,33.475336
6,380.50,7.212632e+05,6.488101e+06,nappe libre,2.637996,0.576275,2.121514,0.627481,377.56303
7,9.00,3.840642e+05,6.306613e+06,nappe libre et captive (spatialement),2.791928,0.650282,2.269965,0.643823,10.654531
8,64.00,7.294586e+05,6.814733e+06,nappe libre,1.894353,0.544618,2.357841,0.697296,60.917017
9,82.00,7.993085e+05,6.872380e+06,nappe libre,2.524803,0.638304,2.068348,0.605236,79.812854


In [148]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

X_train, X_test, y_train, y_test = train_test_split(
    attribute.drop(columns=["état", "water_level_mean"]), attribute["water_level_mean"], test_size=0.2, random_state=42
)

# Fit PCA
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

pca = PCA(n_components=0.95)  # Garde 95% de la variance
X_train_pca = pca.fit_transform(X_train_scaled)

model = LinearRegression()
model.fit(X_train_pca, y_train)

#Test PCA
X_test_scaled = scaler.transform(X_test)
X_test_pca = pca.transform(X_test_scaled)
y_pred = model.predict(X_test_pca)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(mse)
print(r2)

107.44666710531801
0.9882452712351628


In [100]:
def get_static_attributes(bss_id):
    #Extract usefull data from piezo_characs
    raw_df_piezo_characs = pd.read_csv('dataset/static_attributes/piezo_characs.csv', sep=';', encoding='latin-1')
    df_piezo_characs = raw_df_piezo_characs[["BSS_ID", "H", "XL93", "YL93", "formation", "état", "nature", "milieu", "thème", "limnit", "rr_mean", "rr_p", "etp_mean", "stress_p"]]
    
    df_static_attributes_geo =  pd.read_csv('dataset/static_attributes/CAMELS_FR_geology_attributes.csv', sep=';')
    df_static_attributes_hydrogeo =  pd.read_csv('dataset/static_attributes/CAMELS_FR_hydrogeology_attributes.csv', sep=';')
    df_static_attributes_hydrolog =  pd.read_csv('dataset/static_attributes/CAMELS_FR_hydrological_signatures.csv', sep=';')
    df_static_attributes_topo =  pd.read_csv('dataset/static_attributes/CAMELS_FR_topography_general_attributes.csv', sep=';')

    #Extract usefull data from CAMELS_FR_soil_general_attributes
    raw_df_g =  pd.read_csv('dataset/static_attributes/CAMELS_FR_soil_general_attributes.csv', sep=';')
    df_static_attributes_general = raw_df_g[["sta_code_h3"]].copy()
    for sol_agg_level in ["no", "top_subsoil", "topsoil"] :
        for sol_stat in ["mean", "skewness", "na_pct"]:
            df_tmp = raw_df_g[(raw_df_g["sol_stat"] == sol_stat) & (raw_df_g["sol_agg_level"] == sol_agg_level)]
            if sol_stat == "na_pct":
                if sol_stat != "no":
                    df_static_attributes_general['sol_conductivity_'+sol_agg_level+"_"+sol_stat] = raw_df_g[(raw_df_g["sol_stat"] == sol_stat) & (raw_df_g["sol_agg_level"] == sol_agg_level)]["sol_conductivity"]
                    if sol_stat == "top_subsoil":
                        df_static_attributes_general['sol_tawc_inrae_'+sol_agg_level+"_"+sol_stat] = raw_df_g[(raw_df_g["sol_stat"] == sol_stat) & (raw_df_g["sol_agg_level"] == sol_agg_level)]["sol_tawc_inrae"]
            else:
                if sol_agg_level == "no":
                    df_static_attributes_general['sol_depth_to_root_'+sol_stat] = df_tmp["sol_depth_to_root"]
                    df_static_attributes_general['sol_depth_to_bedrock_'+sol_stat] = df_tmp["sol_depth_to_bedrock"]
                else :
                    df_static_attributes_general['sol_clay_'        +sol_agg_level+"_"+sol_stat] = raw_df_g[(raw_df_g["sol_stat"] == sol_stat) & (raw_df_g["sol_agg_level"] == sol_agg_level)]["sol_clay"]
                    df_static_attributes_general['sol_sand_'        +sol_agg_level+"_"+sol_stat] = raw_df_g[(raw_df_g["sol_stat"] == sol_stat) & (raw_df_g["sol_agg_level"] == sol_agg_level)]["sol_sand"]
                    df_static_attributes_general['sol_silt_'        +sol_agg_level+"_"+sol_stat] = raw_df_g[(raw_df_g["sol_stat"] == sol_stat) & (raw_df_g["sol_agg_level"] == sol_agg_level)]["sol_silt"]
                    df_static_attributes_general['sol_oc_'          +sol_agg_level+"_"+sol_stat] = raw_df_g[(raw_df_g["sol_stat"] == sol_stat) & (raw_df_g["sol_agg_level"] == sol_agg_level)]["sol_oc"]
                    df_static_attributes_general['sol_bd_'          +sol_agg_level+"_"+sol_stat] = raw_df_g[(raw_df_g["sol_stat"] == sol_stat) & (raw_df_g["sol_agg_level"] == sol_agg_level)]["sol_bd"]
                    df_static_attributes_general['sol_gravel_'      +sol_agg_level+"_"+sol_stat] = raw_df_g[(raw_df_g["sol_stat"] == sol_stat) & (raw_df_g["sol_agg_level"] == sol_agg_level)]["sol_clay"]
                    df_static_attributes_general['sol_tawc_'        +sol_agg_level+"_"+sol_stat] = raw_df_g[(raw_df_g["sol_stat"] == sol_stat) & (raw_df_g["sol_agg_level"] == sol_agg_level)]["sol_tawc"]
                    df_static_attributes_general['sol_conductivity_'+sol_agg_level+"_"+sol_stat] = raw_df_g[(raw_df_g["sol_stat"] == sol_stat) & (raw_df_g["sol_agg_level"] == sol_agg_level)]["sol_conductivity"]
                    if sol_agg_level == "top_subsoil":
                        df_static_attributes_general['sol_tawc_inrae_'+sol_agg_level+"_"+sol_stat] = raw_df_g[(raw_df_g["sol_stat"] == sol_stat) & (raw_df_g["sol_agg_level"] == sol_agg_level)]["sol_tawc_inrae"]

    df_static_attributes_general = df_static_attributes_general.groupby('sta_code_h3').sum()
    

    # Merge all
    df_all_attributes = pd.merge(df_piezo_characs, df_static_attributes_geo, left_on="limnit", right_on="sta_code_h3", how="left")
    df_all_attributes = pd.merge(df_all_attributes, df_static_attributes_hydrogeo, on="sta_code_h3", how="left")
    df_all_attributes = pd.merge(df_all_attributes, df_static_attributes_hydrolog, on="sta_code_h3", how="left")
    df_all_attributes = pd.merge(df_all_attributes, df_static_attributes_topo, on="sta_code_h3", how="left")
    df_all_attributes = pd.merge(df_all_attributes, df_static_attributes_general, on="sta_code_h3", how="left")
    df_all_attributes.dropna(inplace=True)

    #df_all_attributes = df_all_attributes.assign(water_level_mean=lambda x: water_level_mean(x['BSS_ID'].iloc[0]))
    df_all_attributes["water_level_mean"] = df_all_attributes.apply(water_level_mean, axis=1)
    df_all_attributes.dropna(inplace=True, subset=['water_level_mean'])
    df_all_attributes.reset_index(drop=True, inplace=True)

    return df_all_attributes.drop(columns=['limnit'])

get_static_attributes("BSS000KQJF")

,BSS_ID,H,XL93,YL93,formation,état,nature,milieu,thème,rr_mean,rr_p,etp_mean,stress_p,sta_code_h3,geo_dom_class,geo_su,geo_ss,geo_py,geo_sm,geo_sc,geo_ev,geo_va,geo_vi,geo_vb,geo_pa,geo_pi,geo_pb,geo_mt,geo_wb,geo_ig,geo_nd,hgl_krs_not_karstic,hgl_krs_karstic,hgl_krs_unknown,hgl_thm_alluvial,hgl_thm_sedimentary,hgl_thm_bedrock,hgl_thm_intense_folded,hgl_thm_volcanism,hgl_thm_unknown,hgl_permeability,hgl_porosity,hyd_q_mean,hyd_q_mean_yr,hyd_stream_elas,hyd_slope_fdc,hyd_bfi_ladson,hyd_bfi_lfstat,hyd_bfi_pelletier_pet_ou,hyd_hfd_mean,hyd_q_freq_high,hyd_q_dur_high,hyd_q_freq_low,hyd_q_dur_low,hyd_q_freq_zero,hyd_q_max,hyd_q_date_max,hyd_q_qmna_min,hyd_q_date_qmna,top_altitude_mean,top_slo_mean,top_dist_outlet_mean,top_itopo_mean,top_slo_ori_n,top_slo_ori_ne,top_slo_ori_e,top_slo_ori_se,top_slo_ori_s,top_slo_ori_sw,top_slo_ori_w,top_slo_ori_nw,top_drainage_density,top_mor_form_factor_horton,top_mor_form_factor_square,top_mor_shape_factor,top_mor_compact_coef,top_mor_circ_ratio,top_mor_elong_ratio_circ,top_mor_elong_ratio_catchment,top_mor_relief_ratio,top_slo_flat,top_slo_gentle,top_slo_moderate,top_slo_strong,top_slo_steep,top_slo_very_steep,sol_depth_to_root_mean,sol_depth_to_bedrock_mean,sol_depth_to_root_skewness,sol_depth_to_bedrock_skewness,sol_conductivity_no_na_pct,sol_clay_top_subsoil_mean,sol_sand_top_subsoil_mean,sol_silt_top_subsoil_mean,sol_oc_top_subsoil_mean,sol_bd_top_subsoil_mean,sol_gravel_top_subsoil_mean,sol_tawc_top_subsoil_mean,sol_conductivity_top_subsoil_mean,sol_tawc_inrae_top_subsoil_mean,sol_clay_top_subsoil_skewness,sol_sand_top_subsoil_skewness,sol_silt_top_subsoil_skewness,sol_oc_top_subsoil_skewness,sol_bd_top_subsoil_skewness,sol_gravel_top_subsoil_skewness,sol_tawc_top_subsoil_skewness,sol_conductivity_top_subsoil_skewness,sol_tawc_inrae_top_subsoil_skewness,sol_conductivity_top_subsoil_na_pct,sol_clay_topsoil_mean,sol_sand_topsoil_mean,sol_silt_topsoil_mean,sol_oc_topsoil_mean,sol_bd_topsoil_mean,sol_gravel_topsoil_mean,sol_tawc_topsoil_mean,sol_conductivity_topsoil_mean,sol_clay_topsoil_skewness,sol_sand_topsoil_skewness,sol_silt_topsoil_skewness,sol_oc_topsoil_skewness,sol_bd_topsoil_skewness,sol_gravel_topsoil_skewness,sol_tawc_topsoil_skewness,sol_conductivity_topsoil_skewness,sol_conductivity_topsoil_na_pct,water_level_mean
0,BSS002FEUA,116.00,1.036809e+06,6.312558e+06,Alluvions,nappe libre,unité aquifère,poreux,alluvial,3.074136,0.444559,2.614224,0.757657,Y643401001,sm,0.00,0.00,0.0,61.91,38.09,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,79.99,20.01,0.0,0.00,20.00,0.00,80.00,0.0,0.00,-11.42,0.15,1.38,503.398,1.7336,2.6909,0.58945,0.57403,0.45770,142.35,9.49597,2.66875,26.79775,12.95699,0.0000,62.29,1970-01-12,4.53,1990-09,922.0,19.16,28.97,12.0,23.851,8.731,8.157,15.864,19.487,10.220,4.629,9.061,0.50999,0.1170,0.3048,8.5456,2.0438,0.2394,0.3860,0.3421,0.0283,0.900,5.108,8.344,28.139,46.172,11.337,61.3265,0.7623,1.1818,6.1262,0.0,21.5998,38.7318,39.6684,1.2739,1.3747,21.5998,97.4164,0.2370,34.8534,-0.0368,1.1470,-0.9724,0.1501,-1.3851,-0.0368,0.7141,2.1954,-0.0646,0.0000,20.8097,39.1975,39.9928,1.4153,1.4000,20.8097,51.2080,0.3249,0.3120,1.3325,-1.0602,0.0724,-0.5103,0.3120,0.3831,1.1184,0.0000,99.043689
1,BSS002DWPN,353.00,9.542392e+05,6.321313e+06,Alluvions,nappe libre,unité aquifère,poreux,alluvial,2.682148,0.455382,2.501117,0.713338,X142000101,sm,0.00,0.00,0.0,62.02,37.98,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,97.58,2.42,0.0,0.00,2.42,0.00,97.58,0.0,0.00,-11.74,0.15,0.97,354.271,2.0497,3.2643,0.55353,0.54471,0.24668,150.03,9.65755,2.69853,76.68145,13.12613,0.0000,35.40,1999-10-21,1.92,2017-08,1136.0,18.08,23.07,11.9,11.099,15.715,14.396,9.006,10.165,16.088,12.866,10.666,0.43388,0.2798,0.3899,3.5735,1.8071,0.3062,0.5969,0.5290,0.0452,0.891,4.813,7.883,32.582,46.608,7.223,66.7867,0.7749,1.1060,5.3591,0.0,21.4840,38.0422,40.4738,1.2663,1.3509,21.4840,108.1223,0.2842,38.7019,-0.0364,0.9998,-0.9753,0.0860,-0.4455,-0.0364,0.6616,1.5143,-0.1727,0.0000,20.7398,38.5014,40.7588,1.4